In [11]:
import numpy as np
# import matplotlib.pyplot as plt
# import pandas as pd
import ctypes
from ctypes import c_double, c_int, Structure


In [12]:
# Load the shared library
# lib = ctypes.CDLL('./libvehiclemodel.so')

In [13]:
# Define the function prototype if necessary
# For example, if your function is named `updateVehicleState` and takes a pointer to VehicleState and a double dt
# updateVehicleState = lib.updateVehicleState
# updateVehicleState.argtypes = [ctypes.POINTER(VehicleState), c_double]
# updateVehicleState.restype = None

In [14]:
class VehicleState(Structure):
    _fields_ = [("x", c_double),
                ("y", c_double),
                ("yaw", c_double),
                ("v_x", c_double),
                ("v_y", c_double),
                ("r", c_double),
                ("a_x", c_double),
                ("a_y", c_double),
                ("steer", c_double)]


In [15]:
import time

class Vehicle:
    def __init__(self):
        # Set state to 0
        self.state = {
            'x': 0.0, 'y': 0.0, 'yaw': 0.0,
            'v_x': 0.0, 'v_y': 0.0, 'r': 0.0,
            'a_x': 0.0, 'a_y': 0.0, 'steer': 0.0,
        }
        # Initialize your dynamics model here
        # For example: self.model = YourSimulinkModel()




        # // Initialises some parameters of the model
        # real_T vx = state_.v_x;
        # real_T omega_wheel = vx / wheel_radius_Gain_;  // car_dynamics_P.wheel_radius_Gain = 0.235
        # RT_MODEL_vehicle_and_lowlevel_control_T *rtm = sl_car.getRTM();
        # rtm->contStates->car_CSTATE.x_CSTATE = state_.x;              // x
        # rtm->contStates->car_CSTATE.y_CSTATE = state_.y;              // y
        # rtm->contStates->car_CSTATE.phi_CSTATE = state_.yaw;          // phi
        # rtm->contStates->car_CSTATE.Velocity_CSTATE[0] = vx;          // vx
        # rtm->contStates->car_CSTATE.Velocity_CSTATE[1] = state_.v_y;  // vy
        # rtm->contStates->car_CSTATE.Rotation_CSTATE[2] = state_.r;    // r
        # // rtm->contStates->LinearapproxfromsysID_CSTATE = ; //steering (2 states)
        # rtm->contStates->car_CSTATE.omega_FL_CSTATE = omega_wheel;
        # rtm->contStates->car_CSTATE.omega_FR_CSTATE = omega_wheel;
        # rtm->contStates->car_CSTATE.omega_RL_CSTATE = omega_wheel;
        # rtm->contStates->car_CSTATE.omega_RR_CSTATE = omega_wheel;

        # // Set time state to 0
        # time_last_cmd_ = 0.0;
        # total_sim_time = 0.0;
        # last_step = 0;




    def update_dynamics(self, dt, input_command):
        # Here you would call your Simulink model to update the vehicle's state
        # For example: self.model.step(input_command)
        # Then update the self.state dictionary with the new state
        
        # Placeholder for dynamics update
        self.state['x'] += self.state['v_x'] * dt
        self.state['y'] += self.state['v_y'] * dt
        # Add more sophisticated dynamics update logic here
        




        # void Vehicle::integrateDynamics(State &x, const can_msgs::CarCommand &u, const double dt) {
        # total_sim_time += dt;
        # double arg_VE_vx, arg_VE_vy, arg_VE_ax, arg_VE_ay, arg_VE_r, arg_steering_rad, arg_x, arg_y,
        #     arg_phi;

        # real_T arg_wheel_speed[4];
        # real_T arg_wheel_speed_max[4];
        # real_T arg_wheel_speed_min[4];
        # real_T arg_wheel_torque[4];
        # real_T arg_wheel_torque_target[4];
        # real_T arg_fz[4];
        # real_T arg_yaw_moment_target, arg_steering_rate, arg_DV_DC, arg_DV_delta, arg_steering_torque;
        # bool updated = false;
        # int64_t target_step = total_sim_time / 0.001;
        # for (; last_step < target_step; last_step++) {
        #     if (tuning_.param_key) {
        #     ROS_INFO("New value for param %d: %f", tuning_.param_key, tuning_.new_param_value);
        #     }
        #     sl_car.step(tuning_.param_key, tuning_.new_param_value, u.throttle, u.steering, u.a_x[0],
        #                 u.a_x[1], u.a_x[2], u.steering_angle[0], u.steering_angle[1], u.steering_angle[2],
        #                 u.yaw_rate[0], u.yaw_rate[1], u.yaw_rate[2], u.v_x[0], u.v_x[1], u.v_x[2], u.v_y[0],
        #                 u.v_y[1], u.v_y[2], u.Fx_fl[0], u.Fx_fl[1], u.Fx_fl[2], u.Fx_fr[0], u.Fx_fr[1],
        #                 u.Fx_fr[2], u.Fx_rl[0], u.Fx_rl[1], u.Fx_rl[2], u.Fx_rr[0], u.Fx_rr[1], u.Fx_rr[2],
        #                 arg_x, arg_y, arg_phi, arg_VE_vx, arg_VE_vy, arg_VE_r, arg_VE_ax, arg_VE_ay, arg_fz,
        #                 arg_steering_rad, arg_steering_rate, arg_steering_torque, arg_wheel_speed,
        #                 arg_wheel_speed_max, arg_wheel_speed_min, arg_wheel_torque, arg_wheel_torque_target,
        #                 arg_DV_DC, arg_DV_delta, arg_yaw_moment_target);
        #     updated = true;
        # }
        # if (updated) {
        #     llc_info_.header.stamp = u.header.stamp;
        #     wheels_info_.header.stamp = u.header.stamp;
        #     x.v_x = arg_VE_vx;
        #     x.v_y = arg_VE_vy;
        #     x.a_x = arg_VE_ax;
        #     x.a_y = arg_VE_ay;
        #     x.r = arg_VE_r;
        #     x.steer = arg_steering_rad;
        #     x.x = arg_x;
        #     x.y = arg_y;
        #     x.yaw = arg_phi;

        #     llc_info_.driver_command = arg_DV_DC;
        #     llc_info_.steering_angle_target = arg_DV_delta;
        #     llc_info_.steering_rate = arg_steering_rate;
        #     llc_info_.steering_torque = arg_steering_torque;
        #     llc_info_.yaw_moment_target = arg_yaw_moment_target;
        #     std::copy(std::begin(arg_fz), std::end(arg_fz), std::begin(wheels_info_.wheel_load));
        #     std::copy(std::begin(arg_wheel_speed_max), std::end(arg_wheel_speed_max),
        #             std::begin(wheels_info_.wheel_speed_max));
        #     std::copy(std::begin(arg_wheel_speed_min), std::end(arg_wheel_speed_min),
        #             std::begin(wheels_info_.wheel_speed_min));
        #     std::copy(std::begin(arg_wheel_torque_target), std::end(arg_wheel_torque_target),
        #             std::begin(wheels_info_.wheel_torque_target));

        #     std::copy(std::begin(arg_wheel_speed), std::end(arg_wheel_speed),
        #             std::begin(wheels_info_.motor_speed));

        #     std::copy(std::begin(arg_wheel_torque), std::end(arg_wheel_torque),
        #             std::begin(wheels_info_.motor_torque));

        #     for (int i = 0; i < 4; i++) {
        #     wheels_info_.motor_torque[i] = arg_wheel_torque[i] / gear_ratio_;
        #     wheels_info_.motor_speed[i] = arg_wheel_speed[i] * gear_ratio_;
        #     }
        # }
        # }
    





    def get_state(self):
        return self.state

def main():
    vehicle = Vehicle()
    dt = 0.1  # Time step for the simulation

    try:
        while True:
            # Example command, replace with real command inputs
            input_command = {'throttle': 1.0, 'steering': 0.1}
            vehicle.update_dynamics(dt, input_command)
            print(vehicle.get_state())
            time.sleep(dt)
    except KeyboardInterrupt:
        print("Simulation ended.")


In [16]:
main()

{'x': 0.0, 'y': 0.0, 'yaw': 0.0, 'v_x': 0.0, 'v_y': 0.0, 'r': 0.0, 'a_x': 0.0, 'a_y': 0.0, 'steer': 0.0}
{'x': 0.0, 'y': 0.0, 'yaw': 0.0, 'v_x': 0.0, 'v_y': 0.0, 'r': 0.0, 'a_x': 0.0, 'a_y': 0.0, 'steer': 0.0}
{'x': 0.0, 'y': 0.0, 'yaw': 0.0, 'v_x': 0.0, 'v_y': 0.0, 'r': 0.0, 'a_x': 0.0, 'a_y': 0.0, 'steer': 0.0}
{'x': 0.0, 'y': 0.0, 'yaw': 0.0, 'v_x': 0.0, 'v_y': 0.0, 'r': 0.0, 'a_x': 0.0, 'a_y': 0.0, 'steer': 0.0}
{'x': 0.0, 'y': 0.0, 'yaw': 0.0, 'v_x': 0.0, 'v_y': 0.0, 'r': 0.0, 'a_x': 0.0, 'a_y': 0.0, 'steer': 0.0}
{'x': 0.0, 'y': 0.0, 'yaw': 0.0, 'v_x': 0.0, 'v_y': 0.0, 'r': 0.0, 'a_x': 0.0, 'a_y': 0.0, 'steer': 0.0}
{'x': 0.0, 'y': 0.0, 'yaw': 0.0, 'v_x': 0.0, 'v_y': 0.0, 'r': 0.0, 'a_x': 0.0, 'a_y': 0.0, 'steer': 0.0}
{'x': 0.0, 'y': 0.0, 'yaw': 0.0, 'v_x': 0.0, 'v_y': 0.0, 'r': 0.0, 'a_x': 0.0, 'a_y': 0.0, 'steer': 0.0}
{'x': 0.0, 'y': 0.0, 'yaw': 0.0, 'v_x': 0.0, 'v_y': 0.0, 'r': 0.0, 'a_x': 0.0, 'a_y': 0.0, 'steer': 0.0}
{'x': 0.0, 'y': 0.0, 'yaw': 0.0, 'v_x': 0.0, 'v_y': 0.0